This notebook will be dedicated to training and testing several ML models

In [1]:
from src import data_utils, preprocessing

Only run the cell below by removing the # symbol if you haven't downloaded the zones data

In [4]:
# run this cell to download the required data files containing the trip and zones information

data_utils.download_zones_data('https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip')

Zip file successfully downloaded from 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip' and saved to 'f:\Pf_anyoneAI\Anyone-AI-NYC-Taxis\data\taxi_zones\zones.zip'


In [6]:
# first we get our cleaned data set
df_og = data_utils.clean_trip_data('yellow_tripdata_2022-05.parquet')

In [7]:
df_og.head(3).T

,0,1,2
PULocationID,246,238,163
DOLocationID,151,74,260
fare_amount,17.0,11.0,15.5
improvement_surcharge,0.3,0.3,0.3
congestion_surcharge,2.5,2.5,2.5
airport_fee,0.0,0.0,0.0
travel_time,1122.0,829.0,922.0
time_of_day,night,night,night
day,1,1,1
month,5,5,5


In [8]:
# now we get our X, and ys
X, y_travel_time, y_fare_amount = data_utils.get_feature_target(df_og)

In [9]:
# and we then get our training and testing sets
# we are going to get two sets of y labels, one for travel_time and another for fare_amount
X_train, X_test, y_train_travel_time, y_test_travel_time, y_train_fare_amount, y_test_fare_amount = data_utils.get_train_test_sets(X, y_travel_time, y_fare_amount)

In [10]:
X_train.shape

(2701206, 10)

In [11]:
# we can further split the training set into training and validation
X_train, X_val, y_train_travel_time, y_val_travel_time, y_train_fare_amount, y_val_fare_amount = data_utils.get_train_test_sets(X_train, y_train_travel_time, y_train_fare_amount)

In [12]:
X_train.shape

(2160964, 10)

In [13]:
# now using the training, validation, and testing sets we can preprocess our data
X_train, X_val, X_test = preprocessing.preprocess_data(X_train, X_val, X_test)

Input train data shape:  (2160964, 10)
Input val data shape:  (540242, 10)
Input test data shape:  (675302, 10) 



In [14]:
X_train.shape

(2160964, 12)

In [15]:
column_names_order = ['PULocationID','DOLocationID','improvement_surcharge','congestion_surcharge','airport_fee','day','month','is_weekend','distance_between_zones','morning','afternoon','night']

**Initial Results with no finetuning**

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

In [13]:
# initial run without choosing any new hyperparameters
fare_amount_rf = RandomForestRegressor()

First we will start with RandomForest to predict fare amount

In [14]:
# our initial approach with no fine tuning
fare_amount_rf.fit(X_train,y_train_fare_amount)

RandomForestRegressor()

In [15]:
preds_fare = fare_amount_rf.predict(X_test)
# measure our results
mse = mean_squared_error(y_test_fare_amount, preds_fare)
mae = mean_absolute_error(y_test_fare_amount, preds_fare)
rmse = root_mean_squared_error(y_test_fare_amount, preds_fare)

print(mse)
print(mae)
print(rmse)

24.843686063576
2.253119452425116
4.984344095623415


In [16]:
# from the previous exercise we now get some information

print(f'Estimators: {len(fare_amount_rf.estimators_)}')
for feature_importance,column_name in zip(fare_amount_rf.feature_importances_,column_names_order):
    print(f'Feature {column_name} has an importance of {feature_importance}')

Estimators: 100
Feature PULocationID has an importance of 0.022773891515218588
Feature DOLocationID has an importance of 0.037721428470603884
Feature improvement_surcharge has an importance of 0.0009346294201629111
Feature congestion_surcharge has an importance of 0.004137824382256685
Feature airport_fee has an importance of 0.010204567944246222
Feature day has an importance of 0.02495834098099605
Feature month has an importance of 9.74790463881188e-06
Feature is_weekend has an importance of 0.00279204852880199
Feature distance_between_zones has an importance of 0.8850206141219572
Feature morning has an importance of 0.0026874494862669865
Feature afternoon has an importance of 0.001906561586988242
Feature night has an importance of 0.006852895657862565


Now to predict the travel time

In [17]:
# initial run, no funetuning
travel_time_rf = RandomForestRegressor()

In [18]:
travel_time_rf.fit(X_train, y_train_travel_time)

RandomForestRegressor()

In [19]:
preds_tt = travel_time_rf.predict(X_test)
# measure our results
mse = mean_squared_error(y_test_travel_time, preds_tt)
mae = mean_absolute_error(y_test_travel_time, preds_tt)
rmse = root_mean_squared_error(y_test_travel_time, preds_tt)

print(mse)
print(mae)
print(rmse)

144524.9867117275
234.5827343560056
380.1644206284006


In [20]:
# from the previous exercise we now get some information

print(f'Estimators: {len(travel_time_rf.estimators_)}')
for feature_importance,column_name in zip(travel_time_rf.feature_importances_,column_names_order):
    print(f'Feature {column_name} has an importance of {feature_importance}')

Estimators: 100
Feature PULocationID has an importance of 0.028208844812806787
Feature DOLocationID has an importance of 0.03638914103446075
Feature improvement_surcharge has an importance of 0.0020705118988466357
Feature congestion_surcharge has an importance of 0.005041049813130092
Feature airport_fee has an importance of 0.0047240431082988055
Feature day has an importance of 0.051935559848396616
Feature month has an importance of 0.00013221113970044018
Feature is_weekend has an importance of 0.012235221639949403
Feature distance_between_zones has an importance of 0.7980859715899894
Feature morning has an importance of 0.004786108154745971
Feature afternoon has an importance of 0.0048308437211885625
Feature night has an importance of 0.05156049323848649


**Linear Regresion: Initial response without Fine Tuning.**

In [16]:
from sklearn.linear_model import LinearRegression

In [18]:
#FARE AMOUNT LINEAR REGRESSION
fare_amount_logreg = LinearRegression()
fare_amount_logreg.fit(X_train, y_train_fare_amount)

LinearRegression()

In [21]:
preds_fare_logreg = fare_amount_logreg.predict(X_test)
# measure the results
mse = mean_squared_error(y_test_fare_amount, preds_fare_logreg)
mae = mean_absolute_error(y_test_fare_amount, preds_fare_logreg)
rmse = root_mean_squared_error(y_test_fare_amount, preds_fare_logreg)

print(mse)
print(mae)
print(rmse)

35.18971475390373
2.904904190053445
5.9320919374116015


In [22]:
#TRAVEL TIME LINEAR REGRESSION
travel_time_logreg = LinearRegression()
travel_time_logreg.fit(X_train, y_train_travel_time)

LinearRegression()

In [23]:
preds_travel_logreg = travel_time_logreg.predict(X_test)
# measure the results
mse = mean_squared_error(y_test_travel_time, preds_travel_logreg)
mae = mean_absolute_error(y_test_travel_time, preds_travel_logreg)
rmse = root_mean_squared_error(y_test_travel_time, preds_travel_logreg)

print(mse)
print(mae)
print(rmse)

208218.05153921922
301.46361977293384
456.3091622345745


Now we start finetuning our RandomForest models

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
fare_amount_rf = RandomForestRegressor()

In [ ]:
# we will choose from the following parameters to perform our cross validation search
rf_params = {}

In [ ]:
clf_fare_amount = RandomizedSearchCV(fare_amount_rf,)